<a href="https://colab.research.google.com/github/sweonurulu/colab_notebooks/blob/main/muzayede2_ysa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import requests
import pandas as pd
import numpy as np
import re
import datetime
import string
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from snowballstemmer import TurkishStemmer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from PIL import Image
from io import BytesIO
import tensorflow as tf


#pd.set_option('display.max_colwidth', None)
#pd.reset_option('display.max_colwidth')

## GET DATASET

In [55]:
# son kaç günün verisini istiyorsun?
no_of_day = 7

In [56]:
# Verileri al
response = requests.get("http://95.70.184.79:3066/data/get_item_data/{0}".format(no_of_day))

In [57]:
# Yanıtın JSON verisini al
data = response.json()

In [58]:
# JSON verisinden item_data ve auction_data'yı DataFrame'lere dönüştür
item_data = pd.DataFrame(data["item_data"])
auction_data = pd.DataFrame(data["auction_data"])

In [59]:
item_df = item_data.copy()
auction_df = auction_data.copy()

In [60]:
dataset=pd.merge(left=auction_df,right=item_df,how="outer",on="auction_id")

In [61]:
df=dataset.copy()

In [62]:
# Değiştirme işlemi yapılmayacak sütunlar listesi
excluded_columns = ["item_description", "item_subdescription"]

In [63]:
# Belirtilen sütunlar hariç tüm sütunlarda boş string ve tek boşlukları np.nan ile değiştir
df.loc[:, ~df.columns.isin(excluded_columns)] = df.loc[:, ~df.columns.isin(excluded_columns)].replace({"": np.nan, " ": np.nan})

## EDA

In [64]:
df.head()

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price
0,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239741.0,IŞIL ÖZIŞIK,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,IŞIL ÖZIŞIK,6.500 TL,0.0,17.0,0.0,NaN,İstanbul - Galata Kulesi\nKağıt Üzeri Sulu Boy...,NaN
1,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239742.0,Behruz Kuul,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,BEHRUZ KUUL,7.000 TL,0.0,13.0,0.0,NaN,"Evler Serisi\nTuval Üzeri Yağlı Boya\nİmzalı, ...",NaN
2,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239743.0,FİKRET ÖZTÜRK,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,FİKRET ÖZTÜRK,5.000 TL,0.0,6.0,0.0,NaN,"Ana Grup\nTuval Üzeri Yağlı Boya\nİmzalı, 2024...",NaN
3,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239744.0,HAKAN ÖZDİL,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,HAKAN ÖZDİL,5.000 TL,0.0,6.0,0.0,NaN,"Kurt \nTuval Üzeri Akrilik\nİmzalı , 2024\nEn ...",NaN
4,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239745.0,MEHMET ENGÜL,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,MEHMET ENGÜL,5.000 TL,0.0,4.0,0.0,NaN,"Ortaköy\nTuval Üzeri Yağlı Boya\nİmzalı, 2024\...",NaN


In [65]:
df.shape

(17818, 17)

In [66]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
auction_id,17818.0,9.754720e+07,1.013622e+08,4023974.0,7.023950e+06,6.002417e+07,1.610239e+08,3.560241e+08
item_id,17812.0,1.729240e+11,8.156985e+11,40239741.0,4.024112e+09,2.402402e+10,9.502406e+10,6.302384e+12
item_lot_no,17812.0,5.493029e+02,2.463037e+03,1.0,7.700000e+01,1.750000e+02,3.390000e+02,1.930500e+04
item_sold,17812.0,5.363800e-01,4.986887e-01,0.0,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
item_bid_no,17812.0,3.267460e+00,5.581959e+00,0.0,0.000000e+00,1.000000e+00,4.000000e+00,6.900000e+01


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17818 entries, 0 to 17817
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   auction_id           17818 non-null  int64  
 1   auction_account      17818 non-null  object 
 2   auction_description  17818 non-null  object 
 3   auction_datetime     17818 non-null  object 
 4   auction_url          17818 non-null  object 
 5   item_id              17812 non-null  float64
 6   item_name            17812 non-null  object 
 7   item_lot_no          17812 non-null  float64
 8   item_img             17812 non-null  object 
 9   item_description     17812 non-null  object 
 10  item_reserve_price   17812 non-null  object 
 11  item_sold            17812 non-null  float64
 12  item_tracking_no     13841 non-null  object 
 13  item_bid_no          17812 non-null  float64
 14  item_price           9554 non-null   object 
 15  item_subdescription  17812 non-null 

In [68]:
df.isnull().sum()

auction_id                 0
auction_account            0
auction_description        0
auction_datetime           0
auction_url                0
item_id                    6
item_name                  6
item_lot_no                6
item_img                   6
item_description           6
item_reserve_price         6
item_sold                  6
item_tracking_no        3977
item_bid_no                6
item_price              8264
item_subdescription        6
estimate_price         17515
dtype: int64

In [69]:
df.nunique()

auction_id                69
auction_account           49
auction_description       67
auction_datetime          39
auction_url               69
item_id                17812
item_name                836
item_lot_no             1329
item_img               17812
item_description       13794
item_reserve_price       314
item_sold                  2
item_tracking_no          62
item_bid_no               47
item_price               395
item_subdescription     9563
estimate_price           100
dtype: int64

## CATEGORIZATION

 ### item_name' a göre kategorilendirme

In [80]:
df.dropna(subset=['item_id'], inplace=True)

In [79]:
df[df["item_id"].isnull()]

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price
597,4024031,Janus Mezat,Geçmiş Zamanlar Kartpostal & Efemera Müzayedesi,2024-05-21T22:02:00,https://www.muzayedeapp.com/muzayede/gecmis-za...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2669,4024104,Janus Mezat,Paşa Madalya & Rozet Müzayedesi,2024-05-21T21:17:00,https://www.muzayedeapp.com/muzayede/pasa-mada...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3430,4024123,Janus Mezat,Elite Yabancı Kağıt Para Müzayedesi,2024-05-21T21:02:00,https://www.muzayedeapp.com/muzayede/elite-yab...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6725,31024098,Bit Mezat,"Zengin;Efemera,Kartpostal ve Rozet Müzayedesi",2024-05-21T21:02:00,https://www.muzayedeapp.com/muzayede/zengin-ef...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13205,158024120,Berussa Antika,OSMANLI VE AVRUPA KARMA ESERLER MÜZAYEDESİ,2024-05-21T20:01:00,https://www.muzayedeapp.com/muzayede/osmanli-v...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14629,177023987,Antikalarım | Online Müzayede,"66- ""Timuçin Tanarslan (1943-2015)"" Tablo Müza...",2024-05-21T21:00:00,https://www.muzayedeapp.com/muzayede/66-timuci...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
# Müzayede açıklamaları için Kategorileri belirleyen kelimeler ve bu kelimelere karşılık gelen kategoriler
categories_item_name = {
    'obje': 'obje',

    'efemera' : 'efemera',
    'kartpostal' : 'efemera',
    'pul' : 'efemera',
    'mnh' : 'efemera',
    'fdc' : 'efemera',
    'filateli' : "efemera",
    'fi̇lateli̇' : "efemera",

    'tablo' : 'eser',
    'resim' : 'eser',
    'heykel' : 'eser',

    'para': 'nümismatik',
    'banknot': 'nümismatik',

    'kitap' : 'kitap',
    'dergi' : 'kitap',
    'çizgi roman' : 'kitap'
}

In [82]:
def categorize_item_name(description):
    description = description.lower()  # Küçük harfe dönüştürme
    for keyword, category in categories_item_name.items():
        if keyword in description:
            return category
        else:
            return np.nan

In [83]:
df['item_category'] = df['item_name'].apply(categorize_item_name)

In [84]:
df.head()

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price,item_category
0,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239741.0,IŞIL ÖZIŞIK,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,IŞIL ÖZIŞIK,6.500 TL,0.0,17.0,0.0,NaN,İstanbul - Galata Kulesi\nKağıt Üzeri Sulu Boy...,NaN,NaN
1,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239742.0,Behruz Kuul,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,BEHRUZ KUUL,7.000 TL,0.0,13.0,0.0,NaN,"Evler Serisi\nTuval Üzeri Yağlı Boya\nİmzalı, ...",NaN,NaN
2,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239743.0,FİKRET ÖZTÜRK,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,FİKRET ÖZTÜRK,5.000 TL,0.0,6.0,0.0,NaN,"Ana Grup\nTuval Üzeri Yağlı Boya\nİmzalı, 2024...",NaN,NaN
3,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239744.0,HAKAN ÖZDİL,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,HAKAN ÖZDİL,5.000 TL,0.0,6.0,0.0,NaN,"Kurt \nTuval Üzeri Akrilik\nİmzalı , 2024\nEn ...",NaN,NaN
4,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239745.0,MEHMET ENGÜL,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,MEHMET ENGÜL,5.000 TL,0.0,4.0,0.0,NaN,"Ortaköy\nTuval Üzeri Yağlı Boya\nİmzalı, 2024\...",NaN,NaN


In [85]:
df.isnull().sum()

auction_id                 0
auction_account            0
auction_description        0
auction_datetime           0
auction_url                0
item_id                    0
item_name                  0
item_lot_no                0
item_img                   0
item_description           0
item_reserve_price         0
item_sold                  0
item_tracking_no        3971
item_bid_no                0
item_price              8258
item_subdescription        0
estimate_price         17509
item_category          16308
dtype: int64

 ### auction_description'a göre kategorilendirme

In [86]:
# Müzayede açıklamaları için Kategorileri belirleyen kelimeler ve bu kelimelere karşılık gelen kategoriler
categories = {
    'para': 'nümismatik',
    'banknot': 'nümismatik',
    'nümismatik': 'nümismatik',
    'nümi̇smati̇k' : "nümismatik",
    'numismatik' : 'nümismatik',
    'numi̇smati̇k' : 'nümismatik',
    'nümizmatik' : 'nümismatik',
    'kağit' : 'nümismatik',

    'fdc' : 'efemera',
    'pul' : 'efemera',
    'kartpostal' : 'efemera',
    'filateli' : "efemera",
    'fi̇lateli̇' : "efemera",
    'efemera' : 'efemera',
    'fila' : 'efemera',

    'obje': 'obje',

    'resim' : 'eser',
    'heykel' : 'eser',
    'tablo' : 'eser',
    'eser' : 'eser',
    'sanat' : 'eser',
    'resi̇m' : 'eser',
    'antika' : 'eser',
    'art' : 'eser',

    'kitap' : 'kitap',
    'kitabhane' : 'kitap',
    'çi̇zgi̇ roman' : 'kitap',
    'çizgi roman' : 'kitap',
    'çi̇zgi roman' : 'kitap',
    'çizgi roman' : 'kitap',
    'dergi' : 'kitap',
    'ki̇tap' : 'kitap'
}

In [87]:
def categorize_auctions(description):
    description = description.lower()  # Küçük harfe dönüştürme
    for keyword, category in categories.items():
        if keyword in description:
            return category
    return 'diğer'

In [88]:
df['item_category'].fillna(df['auction_description'].apply(categorize_auctions), inplace=True)

In [89]:
df["item_category"].value_counts()

item_category
nümismatik    4754
efemera       4420
diğer         3498
eser          3176
obje          1576
kitap          388
Name: count, dtype: int64

In [90]:
pd.crosstab(df["item_category"], df["item_sold"])

item_sold,0.0,1.0
item_category,,
diğer,1795,1703
efemera,2472,1948
eser,1611,1565
kitap,56,332
nümismatik,1530,3224
obje,794,782


In [91]:
df.head()

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price,item_category
0,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239741.0,IŞIL ÖZIŞIK,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,IŞIL ÖZIŞIK,6.500 TL,0.0,17.0,0.0,NaN,İstanbul - Galata Kulesi\nKağıt Üzeri Sulu Boy...,NaN,eser
1,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239742.0,Behruz Kuul,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,BEHRUZ KUUL,7.000 TL,0.0,13.0,0.0,NaN,"Evler Serisi\nTuval Üzeri Yağlı Boya\nİmzalı, ...",NaN,eser
2,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239743.0,FİKRET ÖZTÜRK,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,FİKRET ÖZTÜRK,5.000 TL,0.0,6.0,0.0,NaN,"Ana Grup\nTuval Üzeri Yağlı Boya\nİmzalı, 2024...",NaN,eser
3,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239744.0,HAKAN ÖZDİL,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,HAKAN ÖZDİL,5.000 TL,0.0,6.0,0.0,NaN,"Kurt \nTuval Üzeri Akrilik\nİmzalı , 2024\nEn ...",NaN,eser
4,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239745.0,MEHMET ENGÜL,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,MEHMET ENGÜL,5.000 TL,0.0,4.0,0.0,NaN,"Ortaköy\nTuval Üzeri Yağlı Boya\nİmzalı, 2024\...",NaN,eser


In [92]:
df.isnull().sum()

auction_id                 0
auction_account            0
auction_description        0
auction_datetime           0
auction_url                0
item_id                    0
item_name                  0
item_lot_no                0
item_img                   0
item_description           0
item_reserve_price         0
item_sold                  0
item_tracking_no        3971
item_bid_no                0
item_price              8258
item_subdescription        0
estimate_price         17509
item_category              0
dtype: int64

### CATEGORY TO INT

In [93]:
dummy_cols = pd.get_dummies(df["item_category"], drop_first=True)

In [94]:
#df = pd.concat([df, dummy_cols], axis=1)

In [95]:
#df.drop("item_category",axis=1,inplace=True)

In [96]:
df.head()

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price,item_category
0,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239741.0,IŞIL ÖZIŞIK,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,IŞIL ÖZIŞIK,6.500 TL,0.0,17.0,0.0,NaN,İstanbul - Galata Kulesi\nKağıt Üzeri Sulu Boy...,NaN,eser
1,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239742.0,Behruz Kuul,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,BEHRUZ KUUL,7.000 TL,0.0,13.0,0.0,NaN,"Evler Serisi\nTuval Üzeri Yağlı Boya\nİmzalı, ...",NaN,eser
2,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239743.0,FİKRET ÖZTÜRK,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,FİKRET ÖZTÜRK,5.000 TL,0.0,6.0,0.0,NaN,"Ana Grup\nTuval Üzeri Yağlı Boya\nİmzalı, 2024...",NaN,eser
3,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239744.0,HAKAN ÖZDİL,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,HAKAN ÖZDİL,5.000 TL,0.0,6.0,0.0,NaN,"Kurt \nTuval Üzeri Akrilik\nİmzalı , 2024\nEn ...",NaN,eser
4,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239745.0,MEHMET ENGÜL,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,MEHMET ENGÜL,5.000 TL,0.0,4.0,0.0,NaN,"Ortaköy\nTuval Üzeri Yağlı Boya\nİmzalı, 2024\...",NaN,eser


## PREPROCESSING

**takip sayısı sütunundaki boş değerleri 0 ile dolduralım.**

In [97]:
df["item_tracking_no"] = df["item_tracking_no"].replace(np.nan,0)

**veritiplerini düzenleyelim.**

In [98]:
df["auction_account"]=df["auction_account"].astype("string")
df["auction_datetime"] = pd.to_datetime(df["auction_datetime"])
df["auction_description"]=df["auction_description"].astype("string")
df["item_name"]=df["item_name"].astype("string")
df["item_img"]=df["item_img"].astype("string")
df["item_description"]=df["item_description"].astype("string")
df["item_reserve_price"]=df["item_reserve_price"].astype("string")
df["item_sold"]=df["item_sold"].astype("int16")
df["item_tracking_no"]=df["item_tracking_no"].astype("uint")
df["item_subdescription"]=df["item_subdescription"].astype("string")

**Tarih sütununu parçalara ayıralım.**

In [99]:
df[["auction_datetime_day", "auction_datetime_month", "auction_datetime_year",
    "auction_datetime_hour", "auction_datetime_minute"]] = df["auction_datetime"].apply(
        lambda dt: pd.Series([dt.day, dt.month, dt.year, dt.hour, dt.minute]))

**USD verileri TL birimine dönüştürelim.**

In [101]:
df = df[df["item_reserve_price"].astype(str).str.endswith("TL")]

**Açılış fiyatı kısmındaki TL ibaresini silelim ve Açılış fiyatı verilerini integer formate çevirelim.**

In [102]:
df["item_reserve_price"] = df["item_reserve_price"].str.replace(".", "").str.replace(" TL", "").astype(int)

**İhtiyaç duymayacağımız sütunları silelim.**

In [103]:
df.drop(["auction_url","auction_id","item_id","estimate_price","item_price","auction_datetime","auction_account","item_name"],axis=1,inplace=True)

**İki DataFrame'deki açıklama ve alt açıklama sütunlarını birleştirerek tek bir sütun oluşturalım.**

In [104]:
df["item_merged_description"] = df['item_description'].str.cat(df['item_subdescription'], sep=' ')

**açıklama ve alt açıklama sütunları ile işimiz kalmadı. onları silelim.**

In [105]:
df.drop(["item_description","item_subdescription"],axis=1,inplace=True)

In [106]:
df.head()

,auction_description,item_lot_no,item_img,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_category,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute,item_merged_description
0,Janus Resim Müzayedesi,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,6500,0,17,0.0,eser,19,5,2024,20,2,IŞIL ÖZIŞIK İstanbul - Galata Kulesi Kağıt Üz...
1,Janus Resim Müzayedesi,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,7000,0,13,0.0,eser,19,5,2024,20,2,BEHRUZ KUUL Evler Serisi Tuval Üzeri Yağlı Bo...
2,Janus Resim Müzayedesi,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0.0,eser,19,5,2024,20,2,FİKRET ÖZTÜRK Ana Grup Tuval Üzeri Yağlı Boya...
3,Janus Resim Müzayedesi,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0.0,eser,19,5,2024,20,2,HAKAN ÖZDİL Kurt Tuval Üzeri Akrilik İmzalı ...
4,Janus Resim Müzayedesi,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,4,0.0,eser,19,5,2024,20,2,MEHMET ENGÜL Ortaköy Tuval Üzeri Yağlı Boya İ...


## NLP

In [107]:
str_cols =["auction_description","item_merged_description"]

**Tüm metinsel verileri küçük harfe dönüştürelim.**

In [108]:
for col in str_cols:
  df[col] = df[col].str.lower()

**Sütunların içindeki noktalama işaretlerini silme işlemi**

In [109]:
punctutations = string.punctuation

In [110]:
for col in str_cols:
    df[col] = df[col].apply(lambda x: ''.join(char if char not in punctutations else ' ' for char in x))

**Türkçe stopwordsleri indirelim.**

In [111]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [112]:
sw = stopwords.words("turkish")

**Stopwordsleri silelim.**

In [113]:
for col in str_cols:
  df[col] = df[col].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))

**Tokenization**

In [114]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [115]:
for col in str_cols:
  df[col] = df[col].apply(word_tokenize)

**Stemming**

In [116]:
stemmer=TurkishStemmer()

In [117]:
for col in str_cols:
  df[col] = df[col].apply(lambda x: [stemmer.stemWord(word) for word in x])

**Vectorization - Word2Vec**

In [118]:
# Word2Vec modelini eğitme
word2vec_model_auction = Word2Vec(df['auction_description'], vector_size=100, window=5, min_count=1, workers=4)
word2vec_model_item = Word2Vec(df['item_merged_description'], vector_size=100, window=5, min_count=1, workers=4)

# Her kelimenin vektörünü almak için bir fonksiyon tanımlama
def get_word_vector(model, word):
    try:
        return model.wv[word]
    except KeyError:
        # Eğer kelime modelde bulunmuyorsa rastgele bir vektör döndür
        return [0] * model.vector_size

# Her kelimenin vektörünü içeren bir sütun oluşturma
df['auction_description_word2vec'] = df['auction_description'].apply(lambda x: [get_word_vector(word2vec_model_auction, word) for word in x])
df['item_description_word2vec'] = df['item_merged_description'].apply(lambda x: [get_word_vector(word2vec_model_item, word) for word in x])

**vektörize ettiğimiz sütunları veriden silelim.**

In [119]:
df.drop(["auction_description","item_merged_description"],axis=1,inplace=True)

In [120]:
df.head()

,item_lot_no,item_img,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_category,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute,auction_description_word2vec,item_description_word2vec
0,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,6500,0,17,0.0,eser,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.008924023, 0.012273171, -0.0088862935, -0..."
1,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,7000,0,13,0.0,eser,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.035186384, 0.013981382, 0.019103996, -0.0..."
2,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0.0,eser,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.04053203, 0.031577796, -0.014802618, -0.0..."
3,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0.0,eser,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.0845131, 0.122659214, -0.061114334, -0.04..."
4,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,4,0.0,eser,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.20448372, 0.32059506, 0.014333363, -0.222..."


## IMAGE PROCESSING

In [121]:
df["item_category"].unique()

array(['eser', 'obje', 'kitap', 'diğer', 'efemera', 'nümismatik'],
      dtype=object)

In [122]:
df_eser = df[df["item_category"]=="eser"]
df_obje = df[df["item_category"]=="obje"]
df_numismatik = df[df["item_category"]=="nümismatik"]
df_kitap = df[df["item_category"]=="kitap"]
df_efemera = df[df["item_category"]=="efemera"]
df_diger = df[df["item_category"]=="diğer"]

In [123]:
df_eser.drop(["item_img","item_category"],axis=1,inplace=True)
df_obje.drop(["item_img","item_category"],axis=1,inplace=True)
df_numismatik.drop(["item_img","item_category"],axis=1,inplace=True)
df_kitap.drop(["item_img","item_category"],axis=1,inplace=True)
df_efemera.drop(["item_img","item_category"],axis=1,inplace=True)
df_diger.drop(["item_img","item_category"],axis=1,inplace=True)

<ipython-input-123-c4c963951ae8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eser.drop(["item_img","item_category"],axis=1,inplace=True)
<ipython-input-123-c4c963951ae8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_obje.drop(["item_img","item_category"],axis=1,inplace=True)
<ipython-input-123-c4c963951ae8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numismatik.drop(["item_img","item_category"],axis=1,inplace=True

In [124]:
#df_eser = df[df["eser"]==True]

In [125]:
def download_and_convert_image(url):
  # URL'den görüntüyü indirme
  response = requests.get(url)
  # İndirilen görüntüyü açma
  img = Image.open(BytesIO(response.content))
  # Görüntüyü NumPy dizisine dönüştürme
  img_array = np.array(img)
  return img_array

In [126]:
# Verileri işlemeden önce görüntülerin boyutunu optimize etme
#df_eser['item_img_data'] = df_eser['item_img'].apply(download_and_convert_image)

In [127]:
df.head()

,item_lot_no,item_img,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_category,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute,auction_description_word2vec,item_description_word2vec
0,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,6500,0,17,0.0,eser,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.008924023, 0.012273171, -0.0088862935, -0..."
1,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,7000,0,13,0.0,eser,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.035186384, 0.013981382, 0.019103996, -0.0..."
2,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0.0,eser,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.04053203, 0.031577796, -0.014802618, -0.0..."
3,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0.0,eser,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.0845131, 0.122659214, -0.061114334, -0.04..."
4,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,4,0.0,eser,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.20448372, 0.32059506, 0.014333363, -0.222..."


## YSA

In [128]:
df_eser.head()

,item_lot_no,item_reserve_price,item_sold,item_tracking_no,item_bid_no,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute,auction_description_word2vec,item_description_word2vec
0,1.0,6500,0,17,0.0,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.008924023, 0.012273171, -0.0088862935, -0..."
1,2.0,7000,0,13,0.0,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.035186384, 0.013981382, 0.019103996, -0.0..."
2,3.0,5000,0,6,0.0,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.04053203, 0.031577796, -0.014802618, -0.0..."
3,4.0,5000,0,6,0.0,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.0845131, 0.122659214, -0.061114334, -0.04..."
4,5.0,5000,0,4,0.0,19,5,2024,20,2,"[[0.0073048603, -0.02537005, 0.28725195, 0.092...","[[-0.20448372, 0.32059506, 0.014333363, -0.222..."


In [129]:
X = df_eser.drop("item_sold",axis=1)
y = df_eser["item_sold"]

In [130]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [131]:
y_train.shape

(2540,)

In [132]:
# Model oluşturma
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((-1, 1, X_train.shape[1])),  # Add a timestep dimension of 1
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [133]:
# Model derleme
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # İki sınıflı çapraz entropi kayıp fonksiyonu
              metrics=['accuracy'])

In [134]:
import numpy as np

def convert_to_np_array(word2vec_column):
    return np.array([np.array(vec) for vec in word2vec_column])

X_train['auction_description_word2vec'] = convert_to_np_array(X_train['auction_description_word2vec'])
X_train['item_description_word2vec'] = convert_to_np_array(X_train['item_description_word2vec'])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2540,) + inhomogeneous part.

In [ ]:
# Model eğitimi
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

In [ ]:
X_train.info()